In [1]:
from nltk.tokenize import word_tokenize

def score_sentence(sentence, difficulty_score, default_score=0):
    tokens = [w.lower() for w in word_tokenize(sentence) if w.isalpha()]
    scores = [difficulty_score.get(token, default_score) for token in tokens]
    return sum(scores) / len(scores) if scores else 0


In [2]:
difficulty_score = {
    "the": 1, "cat": 1, "is": 1, "animal": 1,
    "panda": 1, "photosynthesis": -2, "mitochondria": -3
}

s1 = "The panda is an animal."
s2 = "Photosynthesis occurs in the mitochondria."

print("문장1 점수:", score_sentence(s1, difficulty_score))  # → 높음
print("문장2 점수:", score_sentence(s2, difficulty_score))  # → 낮음


문장1 점수: 0.8
문장2 점수: -0.8


In [4]:
from gensim.models import KeyedVectors

# Google News Word2Vec (300d)
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

toeic_levels = {
    "400-500": ["book", "school", "student", "desk"],
    "600-700": ["manager", "meeting", "schedule", "project"],
    "800-900": ["delegate", "negotiate", "compliance", "strategy"],
    "1000-1100": ["macroeconomics", "sustainability", "procurement", "liquidity"]
}

# 각 점수대별 평균 임베딩 벡터 계산
import numpy as np

level_vectors = {}
for level, words in toeic_levels.items():
    vectors = [model[w] for w in words if w in model]
    if vectors:
        level_vectors[level] = np.mean(vectors, axis=0)

from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

def sentence_vector(sentence):
    tokens = [w.lower() for w in word_tokenize(sentence) if w.isalpha()]
    vectors = [model[w] for w in tokens if w in model]
    return np.mean(vectors, axis=0) if vectors else None

from sklearn.metrics.pairwise import cosine_similarity

def evaluate_toeic_level(sentence):
    s_vec = sentence_vector(sentence)
    if s_vec is None:
        return {}
    results = {}
    for level, lv_vec in level_vectors.items():
        sim = cosine_similarity(s_vec.reshape(1, -1), lv_vec.reshape(1, -1))[0][0]
        results[level] = sim
    return results

FileNotFoundError: [Errno 2] No such file or directory: 'GoogleNews-vectors-negative300.bin'